In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import geopy as geo
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster


In [3]:
hospital_df=pd.read_csv('../data/Payment_and_Value_of_Care-Hospital.csv')

In [70]:
hospital_df

,Facility ID,Facility Name,Address,City/Town,State,ZIP Code,County/Parish,Telephone Number,Payment Measure ID,Payment Measure Name,...,Value of Care Display ID,Value of Care Display Name,Value of Care Category,Value of Care Footnote,Start Date,End Date,Full Address,Coordinates,Latitude,Longitude
0,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_AMI,Payment for heart attack patients,...,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,07/01/2020,06/30/2023,"1108 ROSS CLARK CIRCLE, DOTHAN, AL, 36301","(Southeast Health, 1108, Ross Clark Circle, Do...",31.216107,-85.363086
1,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_HF,Payment for heart failure patients,...,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment,NaN,07/01/2020,06/30/2023,"1108 ROSS CLARK CIRCLE, DOTHAN, AL, 36301","(Southeast Health, 1108, Ross Clark Circle, Do...",31.216107,-85.363086
2,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_PN,Payment for pneumonia patients,...,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Higher Payment,NaN,07/01/2020,06/30/2023,"1108 ROSS CLARK CIRCLE, DOTHAN, AL, 36301","(Southeast Health, 1108, Ross Clark Circle, Do...",31.216107,-85.363086
3,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,...,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Average Payment,NaN,07/01/2020,03/31/2023,"1108 ROSS CLARK CIRCLE, DOTHAN, AL, 36301","(Southeast Health, 1108, Ross Clark Circle, Do...",31.216107,-85.363086
4,10005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,(256) 593-8310,PAYM_30_AMI,Payment for heart attack patients,...,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,07/01/2020,06/30/2023,"2505 U S HIGHWAY 431 NORTH, BOAZ, AL, 35957",None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18455,671300,GRAHAM REGIONAL MEDICAL CENTER,1301 MONTGOMERY ROAD,GRAHAM,TX,76450,YOUNG,(940) 549-3400,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,...,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Average Payment,NaN,07/01/2020,03/31/2023,"1301 MONTGOMERY ROAD, GRAHAM, TX, 76450","(Graham Regional Medical Center, 1301, Montgom...",33.089291,-98.583631
18456,671301,HEMPHILL COUNTY HOSPITAL,1020 S 4TH ST,CANADIAN,TX,79014,HEMPHILL,(806) 323-6622,PAYM_30_AMI,Payment for heart attack patients,...,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Not Available,5.0,07/01/2020,06/30/2023,"1020 S 4TH ST, CANADIAN, TX, 79014","(Hemphill County Hospital, 1020, South 4th Str...",35.903649,-100.388170
18457,671301,HEMPHILL COUNTY HOSPITAL,1020 S 4TH ST,CANADIAN,TX,79014,HEMPHILL,(806) 323-6622,PAYM_30_HF,Payment for heart failure patients,...,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Not Available,5.0,07/01/2020,06/30/2023,"1020 S 4TH ST, CANADIAN, TX, 79014","(Hemphill County Hospital, 1020, South 4th Str...",35.903649,-100.388170
18458,671301,HEMPHILL COUNTY HOSPITAL,1020 S 4TH ST,CANADIAN,TX,79014,HEMPHILL,(806) 323-6622,PAYM_30_PN,Payment for pneumonia patients,...,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Not Available,5.0,07/01/2020,06/30/2023,"1020 S 4TH ST, CANADIAN, TX, 79014","(Hemphill County Hospital, 1020, South 4th Str...",35.903649,-100.388170


In [4]:
#Change ZIP Code column from integer to string
hospital_df['ZIP Code'] =hospital_df['ZIP Code'].astype(str)

In [5]:
#Create a new column which concatenates all of the address info
hospital_df['Full Address']= hospital_df['Address'] +', ' + hospital_df['City/Town'] +', ' + hospital_df['State'] +', ' + hospital_df['ZIP Code']

In [6]:
#Make sure this new column is set to a string datatype
hospital_df['Full Address'] =hospital_df['Full Address'].astype('string')

In [18]:
hospital_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18460 entries, 0 to 18459
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Facility ID                 18460 non-null  int64  
 1   Facility Name               18460 non-null  object 
 2   Address                     18460 non-null  object 
 3   City/Town                   18460 non-null  object 
 4   State                       18460 non-null  object 
 5   ZIP Code                    18460 non-null  object 
 6   County/Parish               18460 non-null  object 
 7   Telephone Number            18460 non-null  object 
 8   Payment Measure ID          18460 non-null  object 
 9   Payment Measure Name        18460 non-null  object 
 10  Payment Category            18460 non-null  object 
 11  Denominator                 18460 non-null  object 
 12  Payment                     18460 non-null  object 
 13  Lower Estimate              184

In [7]:
#Create a sample dataset using the head of the full one, for testing
head=hospital_df.head()

In [10]:
print(head)

   Facility ID                    Facility Name                     Address  \
0        10001  SOUTHEAST HEALTH MEDICAL CENTER      1108 ROSS CLARK CIRCLE   
1        10001  SOUTHEAST HEALTH MEDICAL CENTER      1108 ROSS CLARK CIRCLE   
2        10001  SOUTHEAST HEALTH MEDICAL CENTER      1108 ROSS CLARK CIRCLE   
3        10001  SOUTHEAST HEALTH MEDICAL CENTER      1108 ROSS CLARK CIRCLE   
4        10005         MARSHALL MEDICAL CENTERS  2505 U S HIGHWAY 431 NORTH   

  City/Town State ZIP Code County/Parish Telephone Number Payment Measure ID  \
0    DOTHAN    AL    36301       HOUSTON   (334) 793-8701        PAYM_30_AMI   
1    DOTHAN    AL    36301       HOUSTON   (334) 793-8701         PAYM_30_HF   
2    DOTHAN    AL    36301       HOUSTON   (334) 793-8701         PAYM_30_PN   
3    DOTHAN    AL    36301       HOUSTON   (334) 793-8701   PAYM_90_HIP_KNEE   
4      BOAZ    AL    35957      MARSHALL   (256) 593-8310        PAYM_30_AMI   

                        Payment Measure Name

In [9]:
#import the Nominatim object from geopy, which performs the actual geocoding
from geopy import Nominatim

# Initialize the Nominatim geocoder
geolocator = Nominatim(user_agent="my-geocoder",timeout=10)

In [62]:
#Test the geocoder on a single address
geolocator.geocode('1108 ROSS CLARK CIRCLE, DOTHAN, AL, 36301')

Location(Southeast Health, 1108, Ross Clark Circle, Dothan, Houston County, Alabama, 36301, United States, (31.21610655, -85.36308641535447, 0.0))

In [48]:
#Test the geocoder on the head dataframe (first 5 rows of the full one)
head['Coordinates']=head['Full Address'].apply(geolocator.geocode)

/var/folders/06/x5sf89c509s5h3y417rsqw9c0000gp/T/ipykernel_80588/1946055236.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  head['Coordinates']=head['Facility Name'].apply(geolocator.geocode)


In [49]:
head

,Facility ID,Facility Name,Address,City/Town,State,ZIP Code,County/Parish,Telephone Number,Payment Measure ID,Payment Measure Name,...,Value of Care Display ID,Value of Care Display Name,Value of Care Category,Value of Care Footnote,Start Date,End Date,Full Address,Coordinates,Latitude,Longitude
0,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_AMI,Payment for heart attack patients,...,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,07/01/2020,06/30/2023,"1108 ROSS CLARK CIRCLE, DOTHAN, AL, 36301",(Southeast Georgia Health System - Camden Camp...,31.216107,-85.363086
1,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_HF,Payment for heart failure patients,...,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment,NaN,07/01/2020,06/30/2023,"1108 ROSS CLARK CIRCLE, DOTHAN, AL, 36301",(Southeast Georgia Health System - Camden Camp...,31.216107,-85.363086
2,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_PN,Payment for pneumonia patients,...,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Higher Payment,NaN,07/01/2020,06/30/2023,"1108 ROSS CLARK CIRCLE, DOTHAN, AL, 36301",(Southeast Georgia Health System - Camden Camp...,31.216107,-85.363086
3,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,...,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Average Payment,NaN,07/01/2020,03/31/2023,"1108 ROSS CLARK CIRCLE, DOTHAN, AL, 36301",(Southeast Georgia Health System - Camden Camp...,31.216107,-85.363086
4,10005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,(256) 593-8310,PAYM_30_AMI,Payment for heart attack patients,...,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,07/01/2020,06/30/2023,"2505 U S HIGHWAY 431 NORTH, BOAZ, AL, 35957",None,NaN,NaN


In [29]:
#Apply the latitude method (from geopy) to the coordinates produced by the geocoder to grab the latitude values
head['Latitude']=head['Coordinates'].apply(lambda x: x.latitude if x != None else None)
head

/var/folders/06/x5sf89c509s5h3y417rsqw9c0000gp/T/ipykernel_80588/3713128454.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  head['Latitude']=head['Coordinates'].apply(lambda x: x.latitude if x != None else None)


,Facility ID,Facility Name,Address,City/Town,State,ZIP Code,County/Parish,Telephone Number,Payment Measure ID,Payment Measure Name,...,Payment Footnote,Value of Care Display ID,Value of Care Display Name,Value of Care Category,Value of Care Footnote,Start Date,End Date,Full Address,Coordinates,Latitude
0,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_AMI,Payment for heart attack patients,...,NaN,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,07/01/2020,06/30/2023,"1108 ROSS CLARK CIRCLE, DOTHAN, AL, 36301","(Southeast Health, 1108, Ross Clark Circle, Do...",31.216107
1,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_HF,Payment for heart failure patients,...,NaN,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment,NaN,07/01/2020,06/30/2023,"1108 ROSS CLARK CIRCLE, DOTHAN, AL, 36301","(Southeast Health, 1108, Ross Clark Circle, Do...",31.216107
2,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_PN,Payment for pneumonia patients,...,NaN,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Higher Payment,NaN,07/01/2020,06/30/2023,"1108 ROSS CLARK CIRCLE, DOTHAN, AL, 36301","(Southeast Health, 1108, Ross Clark Circle, Do...",31.216107
3,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,...,NaN,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Average Payment,NaN,07/01/2020,03/31/2023,"1108 ROSS CLARK CIRCLE, DOTHAN, AL, 36301","(Southeast Health, 1108, Ross Clark Circle, Do...",31.216107
4,10005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,(256) 593-8310,PAYM_30_AMI,Payment for heart attack patients,...,NaN,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,07/01/2020,06/30/2023,"2505 U S HIGHWAY 431 NORTH, BOAZ, AL, 35957",None,NaN


In [30]:
#Do the same thing for the longitudes
head['Longitude']=head['Coordinates'].apply(lambda x: x.longitude if x != None else None)
head

/var/folders/06/x5sf89c509s5h3y417rsqw9c0000gp/T/ipykernel_80588/2895598278.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  head['Longitude']=head['Coordinates'].apply(lambda x: x.longitude if x != None else None)


,Facility ID,Facility Name,Address,City/Town,State,ZIP Code,County/Parish,Telephone Number,Payment Measure ID,Payment Measure Name,...,Value of Care Display ID,Value of Care Display Name,Value of Care Category,Value of Care Footnote,Start Date,End Date,Full Address,Coordinates,Latitude,Longitude
0,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_AMI,Payment for heart attack patients,...,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,07/01/2020,06/30/2023,"1108 ROSS CLARK CIRCLE, DOTHAN, AL, 36301","(Southeast Health, 1108, Ross Clark Circle, Do...",31.216107,-85.363086
1,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_HF,Payment for heart failure patients,...,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment,NaN,07/01/2020,06/30/2023,"1108 ROSS CLARK CIRCLE, DOTHAN, AL, 36301","(Southeast Health, 1108, Ross Clark Circle, Do...",31.216107,-85.363086
2,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_PN,Payment for pneumonia patients,...,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Higher Payment,NaN,07/01/2020,06/30/2023,"1108 ROSS CLARK CIRCLE, DOTHAN, AL, 36301","(Southeast Health, 1108, Ross Clark Circle, Do...",31.216107,-85.363086
3,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,...,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Average Payment,NaN,07/01/2020,03/31/2023,"1108 ROSS CLARK CIRCLE, DOTHAN, AL, 36301","(Southeast Health, 1108, Ross Clark Circle, Do...",31.216107,-85.363086
4,10005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,(256) 593-8310,PAYM_30_AMI,Payment for heart attack patients,...,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,07/01/2020,06/30/2023,"2505 U S HIGHWAY 431 NORTH, BOAZ, AL, 35957",None,NaN,NaN


In [77]:
head['GeoAddress']=head['Coordinates'].apply(lambda x: x.address if x != None else None)
head['GeoAddress']

0    Southeast Georgia Health System - Camden Campu...
1    Southeast Georgia Health System - Camden Campu...
2    Southeast Georgia Health System - Camden Campu...
3    Southeast Georgia Health System - Camden Campu...
4                                                 None
Name: GeoAddress, dtype: object


/var/folders/06/x5sf89c509s5h3y417rsqw9c0000gp/T/ipykernel_80588/1792518588.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  head['GeoAddress']=head['Coordinates'].apply(lambda x: x.address if x != None else None)


In [31]:
#Now run the the geocoder on the full dataset (WARNING: THIS TOOK 5 HOURS TO RUN)
hospital_df['Coordinates']=hospital_df['Full Address'].apply(geolocator.geocode)

In [32]:
hospital_df['Latitude']=hospital_df['Coordinates'].apply(lambda x: x.latitude if x != None else None)
hospital_df

,Facility ID,Facility Name,Address,City/Town,State,ZIP Code,County/Parish,Telephone Number,Payment Measure ID,Payment Measure Name,...,Value of Care Display ID,Value of Care Display Name,Value of Care Category,Value of Care Footnote,Start Date,End Date,Full Address,Coordinates,Latitude,Longitude
0,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_AMI,Payment for heart attack patients,...,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,07/01/2020,06/30/2023,"1108 ROSS CLARK CIRCLE, DOTHAN, AL, 36301","(Southeast Health, 1108, Ross Clark Circle, Do...",31.216107,-85.363086
1,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_HF,Payment for heart failure patients,...,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment,NaN,07/01/2020,06/30/2023,"1108 ROSS CLARK CIRCLE, DOTHAN, AL, 36301","(Southeast Health, 1108, Ross Clark Circle, Do...",31.216107,-85.363086
2,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_PN,Payment for pneumonia patients,...,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Higher Payment,NaN,07/01/2020,06/30/2023,"1108 ROSS CLARK CIRCLE, DOTHAN, AL, 36301","(Southeast Health, 1108, Ross Clark Circle, Do...",31.216107,-85.363086
3,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,...,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Average Payment,NaN,07/01/2020,03/31/2023,"1108 ROSS CLARK CIRCLE, DOTHAN, AL, 36301","(Southeast Health, 1108, Ross Clark Circle, Do...",31.216107,-85.363086
4,10005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,(256) 593-8310,PAYM_30_AMI,Payment for heart attack patients,...,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,07/01/2020,06/30/2023,"2505 U S HIGHWAY 431 NORTH, BOAZ, AL, 35957",None,NaN,NaN


In [33]:
hospital_df['Longitude']=hospital_df['Coordinates'].apply(lambda x: x.longitude if x != None else None)
hospital_df

,Facility ID,Facility Name,Address,City/Town,State,ZIP Code,County/Parish,Telephone Number,Payment Measure ID,Payment Measure Name,...,Value of Care Display ID,Value of Care Display Name,Value of Care Category,Value of Care Footnote,Start Date,End Date,Full Address,Coordinates,Latitude,Longitude
0,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_AMI,Payment for heart attack patients,...,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,07/01/2020,06/30/2023,"1108 ROSS CLARK CIRCLE, DOTHAN, AL, 36301","(Southeast Health, 1108, Ross Clark Circle, Do...",31.216107,-85.363086
1,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_HF,Payment for heart failure patients,...,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment,NaN,07/01/2020,06/30/2023,"1108 ROSS CLARK CIRCLE, DOTHAN, AL, 36301","(Southeast Health, 1108, Ross Clark Circle, Do...",31.216107,-85.363086
2,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_PN,Payment for pneumonia patients,...,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Higher Payment,NaN,07/01/2020,06/30/2023,"1108 ROSS CLARK CIRCLE, DOTHAN, AL, 36301","(Southeast Health, 1108, Ross Clark Circle, Do...",31.216107,-85.363086
3,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,...,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Average Payment,NaN,07/01/2020,03/31/2023,"1108 ROSS CLARK CIRCLE, DOTHAN, AL, 36301","(Southeast Health, 1108, Ross Clark Circle, Do...",31.216107,-85.363086
4,10005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,(256) 593-8310,PAYM_30_AMI,Payment for heart attack patients,...,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,07/01/2020,06/30/2023,"2505 U S HIGHWAY 431 NORTH, BOAZ, AL, 35957",None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18455,671300,GRAHAM REGIONAL MEDICAL CENTER,1301 MONTGOMERY ROAD,GRAHAM,TX,76450,YOUNG,(940) 549-3400,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,...,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Average Payment,NaN,07/01/2020,03/31/2023,"1301 MONTGOMERY ROAD, GRAHAM, TX, 76450","(Graham Regional Medical Center, 1301, Montgom...",33.089291,-98.583631
18456,671301,HEMPHILL COUNTY HOSPITAL,1020 S 4TH ST,CANADIAN,TX,79014,HEMPHILL,(806) 323-6622,PAYM_30_AMI,Payment for heart attack patients,...,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Not Available,5.0,07/01/2020,06/30/2023,"1020 S 4TH ST, CANADIAN, TX, 79014","(Hemphill County Hospital, 1020, South 4th Str...",35.903649,-100.388170
18457,671301,HEMPHILL COUNTY HOSPITAL,1020 S 4TH ST,CANADIAN,TX,79014,HEMPHILL,(806) 323-6622,PAYM_30_HF,Payment for heart failure patients,...,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Not Available,5.0,07/01/2020,06/30/2023,"1020 S 4TH ST, CANADIAN, TX, 79014","(Hemphill County Hospital, 1020, South 4th Str...",35.903649,-100.388170
18458,671301,HEMPHILL COUNTY HOSPITAL,1020 S 4TH ST,CANADIAN,TX,79014,HEMPHILL,(806) 323-6622,PAYM_30_PN,Payment for pneumonia patients,...,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Not Available,5.0,07/01/2020,06/30/2023,"1020 S 4TH ST, CANADIAN, TX, 79014","(Hemphill County Hospital, 1020, South 4th Str...",35.903649,-100.388170


In [78]:
hospital_df['GeoAddress']=hospital_df['Coordinates'].apply(lambda x: x.address if x != None else None)
hospital_df['GeoAddress']

0        Southeast Health, 1108, Ross Clark Circle, Dot...
1        Southeast Health, 1108, Ross Clark Circle, Dot...
2        Southeast Health, 1108, Ross Clark Circle, Dot...
3        Southeast Health, 1108, Ross Clark Circle, Dot...
4                                                     None
                               ...                        
18455    Graham Regional Medical Center, 1301, Montgome...
18456    Hemphill County Hospital, 1020, South 4th Stre...
18457    Hemphill County Hospital, 1020, South 4th Stre...
18458    Hemphill County Hospital, 1020, South 4th Stre...
18459    Hemphill County Hospital, 1020, South 4th Stre...
Name: GeoAddress, Length: 18460, dtype: object

In [34]:
#Export the results to a new csv file
hospital_df.to_csv('Payment_and_Value_of_Care-geocoded.csv')

In [35]:
#Check the dataframe now.  There are nearly 4000 records that failed to return coordinates.
hospital_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18460 entries, 0 to 18459
Data columns (total 26 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Facility ID                 18460 non-null  int64  
 1   Facility Name               18460 non-null  object 
 2   Address                     18460 non-null  object 
 3   City/Town                   18460 non-null  object 
 4   State                       18460 non-null  object 
 5   ZIP Code                    18460 non-null  object 
 6   County/Parish               18460 non-null  object 
 7   Telephone Number            18460 non-null  object 
 8   Payment Measure ID          18460 non-null  object 
 9   Payment Measure Name        18460 non-null  object 
 10  Payment Category            18460 non-null  object 
 11  Denominator                 18460 non-null  object 
 12  Payment                     18460 non-null  object 
 13  Lower Estimate              184

In [73]:
#Apparently, geolocators can also be run on facility names instead of addresses!
geolocator.geocode('SHELBY BAPTIST MEDICAL CENTER')

Location(Shelby Baptist Medical Center, 1000, 1st Street North, Alabaster, Shelby County, Alabama, 35007, United States, (33.2526682, -86.81296818785188, 0.0))

In [52]:
#So now, let's repeat the entire process, this time passing facility names to the geocoder instead of addresses.
hospital_df2=pd.read_csv('../data/Payment_and_Value_of_Care-Hospital.csv')

In [53]:
#WARNING, THIS TOOK 5 HOURS
hospital_df2['Coordinates']=hospital_df2['Facility Name'].apply(geolocator.geocode)

In [55]:
hospital_df2['Latitude']=hospital_df2['Coordinates'].apply(lambda x: x.latitude if x != None else None)
hospital_df2['Longitude']=hospital_df2['Coordinates'].apply(lambda x: x.longitude if x != None else None)
hospital_df2

,Facility ID,Facility Name,Address,City/Town,State,ZIP Code,County/Parish,Telephone Number,Payment Measure ID,Payment Measure Name,...,Payment Footnote,Value of Care Display ID,Value of Care Display Name,Value of Care Category,Value of Care Footnote,Start Date,End Date,Coordinates,Latitude,Longitude
0,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_AMI,Payment for heart attack patients,...,NaN,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,07/01/2020,06/30/2023,(Southeast Georgia Health System - Camden Camp...,30.780965,-81.614615
1,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_HF,Payment for heart failure patients,...,NaN,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment,NaN,07/01/2020,06/30/2023,(Southeast Georgia Health System - Camden Camp...,30.780965,-81.614615
2,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_PN,Payment for pneumonia patients,...,NaN,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Higher Payment,NaN,07/01/2020,06/30/2023,(Southeast Georgia Health System - Camden Camp...,30.780965,-81.614615
3,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,...,NaN,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Average Payment,NaN,07/01/2020,03/31/2023,(Southeast Georgia Health System - Camden Camp...,30.780965,-81.614615
4,10005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,(256) 593-8310,PAYM_30_AMI,Payment for heart attack patients,...,NaN,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,07/01/2020,06/30/2023,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18455,671300,GRAHAM REGIONAL MEDICAL CENTER,1301 MONTGOMERY ROAD,GRAHAM,TX,76450,YOUNG,(940) 549-3400,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,...,NaN,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Average Payment,NaN,07/01/2020,03/31/2023,"(Graham Regional Medical Center, 1301, Montgom...",33.089291,-98.583631
18456,671301,HEMPHILL COUNTY HOSPITAL,1020 S 4TH ST,CANADIAN,TX,79014,HEMPHILL,(806) 323-6622,PAYM_30_AMI,Payment for heart attack patients,...,5,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Not Available,5.0,07/01/2020,06/30/2023,"(Hemphill County Hospital, 1020, South 4th Str...",35.903649,-100.388170
18457,671301,HEMPHILL COUNTY HOSPITAL,1020 S 4TH ST,CANADIAN,TX,79014,HEMPHILL,(806) 323-6622,PAYM_30_HF,Payment for heart failure patients,...,5,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Not Available,5.0,07/01/2020,06/30/2023,"(Hemphill County Hospital, 1020, South 4th Str...",35.903649,-100.388170
18458,671301,HEMPHILL COUNTY HOSPITAL,1020 S 4TH ST,CANADIAN,TX,79014,HEMPHILL,(806) 323-6622,PAYM_30_PN,Payment for pneumonia patients,...,5,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Not Available,5.0,07/01/2020,06/30/2023,"(Hemphill County Hospital, 1020, South 4th Str...",35.903649,-100.388170


In [56]:
hospital_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18460 entries, 0 to 18459
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Facility ID                 18460 non-null  int64  
 1   Facility Name               18460 non-null  object 
 2   Address                     18460 non-null  object 
 3   City/Town                   18460 non-null  object 
 4   State                       18460 non-null  object 
 5   ZIP Code                    18460 non-null  int64  
 6   County/Parish               18460 non-null  object 
 7   Telephone Number            18460 non-null  object 
 8   Payment Measure ID          18460 non-null  object 
 9   Payment Measure Name        18460 non-null  object 
 10  Payment Category            18460 non-null  object 
 11  Denominator                 18460 non-null  object 
 12  Payment                     18460 non-null  object 
 13  Lower Estimate              184

In [57]:
hospital_df2.to_csv('Payment_and_Value_of_Care-geocodedfacilityname.csv')

In [60]:
#Next, we merge both of our resulting datasets
merged_hospital_df=pd.merge(hospital_df,hospital_df2, on=['Facility ID','Payment Measure ID'], how='inner')
merged_hospital_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18460 entries, 0 to 18459
Data columns (total 49 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Facility ID                   18460 non-null  int64  
 1   Facility Name_x               18460 non-null  object 
 2   Address_x                     18460 non-null  object 
 3   City/Town_x                   18460 non-null  object 
 4   State_x                       18460 non-null  object 
 5   ZIP Code_x                    18460 non-null  object 
 6   County/Parish_x               18460 non-null  object 
 7   Telephone Number_x            18460 non-null  object 
 8   Payment Measure ID            18460 non-null  object 
 9   Payment Measure Name_x        18460 non-null  object 
 10  Payment Category_x            18460 non-null  object 
 11  Denominator_x                 18460 non-null  object 
 12  Payment_x                     18460 non-null  object 
 13  L

In [61]:
#Now, impute the null coordinate values of the first dataframe with the coordinates from the second
merged_hospital_df['Coordinates_x']=merged_hospital_df['Coordinates_x'].fillna(merged_hospital_df['Coordinates_y'])
merged_hospital_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18460 entries, 0 to 18459
Data columns (total 49 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Facility ID                   18460 non-null  int64  
 1   Facility Name_x               18460 non-null  object 
 2   Address_x                     18460 non-null  object 
 3   City/Town_x                   18460 non-null  object 
 4   State_x                       18460 non-null  object 
 5   ZIP Code_x                    18460 non-null  object 
 6   County/Parish_x               18460 non-null  object 
 7   Telephone Number_x            18460 non-null  object 
 8   Payment Measure ID            18460 non-null  object 
 9   Payment Measure Name_x        18460 non-null  object 
 10  Payment Category_x            18460 non-null  object 
 11  Denominator_x                 18460 non-null  object 
 12  Payment_x                     18460 non-null  object 
 13  L

In [63]:
#Rerun the latitude and longitude methods on the cleaned Coordinates column
merged_hospital_df['Latitude_m']=merged_hospital_df['Coordinates_x'].apply(lambda x: x.latitude if x != None else None)
merged_hospital_df['Longitude_m']=merged_hospital_df['Coordinates_x'].apply(lambda x: x.longitude if x != None else None)
merged_hospital_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18460 entries, 0 to 18459
Data columns (total 51 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Facility ID                   18460 non-null  int64  
 1   Facility Name_x               18460 non-null  object 
 2   Address_x                     18460 non-null  object 
 3   City/Town_x                   18460 non-null  object 
 4   State_x                       18460 non-null  object 
 5   ZIP Code_x                    18460 non-null  object 
 6   County/Parish_x               18460 non-null  object 
 7   Telephone Number_x            18460 non-null  object 
 8   Payment Measure ID            18460 non-null  object 
 9   Payment Measure Name_x        18460 non-null  object 
 10  Payment Category_x            18460 non-null  object 
 11  Denominator_x                 18460 non-null  object 
 12  Payment_x                     18460 non-null  object 
 13  L

In [66]:
#Clean up this merged dataframe, take only the columns you need
final_hospital_df=merged_hospital_df[['Facility ID','Facility Name_x','Address_x','City/Town_x','State_x','ZIP Code_x','County/Parish_x','Telephone Number_x','Payment Measure ID','Payment Measure Name_x','Payment Category_x','Denominator_x','Payment_x','Lower Estimate_x','Higher Estimate_x','Payment Footnote_x','Value of Care Display ID_x','Value of Care Display Name_x','Value of Care Category_x','Value of Care Footnote_x','Start Date_x','End Date_x','Coordinates_x','Latitude_m','Longitude_m']]

In [68]:
final_hospital_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18460 entries, 0 to 18459
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Facility ID                   18460 non-null  int64  
 1   Facility Name_x               18460 non-null  object 
 2   Address_x                     18460 non-null  object 
 3   City/Town_x                   18460 non-null  object 
 4   State_x                       18460 non-null  object 
 5   ZIP Code_x                    18460 non-null  object 
 6   County/Parish_x               18460 non-null  object 
 7   Telephone Number_x            18460 non-null  object 
 8   Payment Measure ID            18460 non-null  object 
 9   Payment Measure Name_x        18460 non-null  object 
 10  Payment Category_x            18460 non-null  object 
 11  Denominator_x                 18460 non-null  object 
 12  Payment_x                     18460 non-null  object 
 13  L

In [69]:
final_hospital_df.to_csv('Payment_and_Value_of_Care-geocoded_final.csv')